# package import & env setting

In [1]:
from langchain_mcp_adapters.client import MultiServerMCPClient
from langgraph.prebuilt import create_react_agent

from langchain_openai import ChatOpenAI


import os
from dotenv import load_dotenv
load_dotenv()



os.environ["OPENWEATHER_API_KEY"] = os.getenv("OPENWEATHER_API_KEY")
model = ChatOpenAI(model="gpt-4o")

# MCP Client test

In [ ]:
weather_server_info = {
        "weather": {
            "url": "http://localhost:7000/sse",
            "transport": "sse",
        }
    }
async with MultiServerMCPClient(weather_server_info) as client:
    weather_tools = client.get_tools()
    agent = create_react_agent(model, client.get_tools())
    weather_response = await agent.ainvoke({"messages": "서울 날씨 알려줄래?"})
    print(weather_response['messages'][-1].content)

서울의 현재 날씨는 다음과 같습니다:

- **온도:** 10.76도
- **날씨:** 맑음
- **풍속:** 1.03m/s

이런 날씨에는 약간 쌀쌀할 수 있으므로, 긴소매 옷이나 가벼운 자켓을 추천합니다. 바람이 약하게 불고 있으니, 외출 시 가벼운 방풍 외투를 갖고 나가시면 좋습니다.


In [14]:
from langgraph.graph import StateGraph, END
from typing import TypedDict

class AgentState(TypedDict):
    input: str
    response: str
    route: str


def goto_weather(state: AgentState) -> AgentState:
    state["route"] = "weather"
    return state

async def make_graph(state: AgentState) -> AgentState:
    async with MultiServerMCPClient(weather_server_info) as client:
        agent = create_react_agent(model, client.get_tools())

        user_input = state["input"]
        weather_response = await agent.ainvoke({"messages": user_input})
        state["response"] = weather_response['messages'][-1].content
        return state

# 그래프 생성
def create_graph():
    graph = StateGraph(AgentState)
    
    graph.add_node("category", goto_weather)
    graph.add_node("weather", make_graph)

    graph.set_entry_point("category")
    graph.add_edge("category", "weather")
    graph.add_edge("weather", END)
    
    return graph.compile()

In [15]:
async def run_graph(user_input: str):
    graph = create_graph()
    result = await graph.ainvoke({"input": user_input,
                                  "response": "",
                                  "route": ""})
    return result

In [18]:
result = await run_graph("서울 날씨 알려줘")
print(result['response'])

서울의 현재 날씨는 맑음이며, 온도는 약 10.76도입니다. 바람은 약간 부는 편으로 풍속은 1.03m/s입니다. 

이런 날씨에는 다음과 같은 복장을 추천드립니다:
- 얇은 니트나 긴 소매 옷과 함께 가벼운 자켓을 입으세요.
- 바람을 막을 수 있는 얇은 스카프나 목도리를 준비하는 것도 좋습니다.
- 편안한 운동화나 부츠를 신는 것이 좋습니다.

이 복장 제안으로 하루를 보다 편안하게 보내세요!
